<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Mistral_7b_hfdeployment_withflashattn2_squad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [ ]:
#!pip install diffusers safetensors  --quiet
#!pip install colab-env --quiet

import colab_env
import os

access_token = os.getenv("HUGGINGFACE_ACCESS_TOKEN")

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")


In [3]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [8]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [5]:
# set device
device = 'cuda'

In [9]:
torch.__version__

'2.1.0+cu121'

In [ ]:
!apt-get update && apt-get install -y cuda-11-0

In [11]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Sat Mar  9 13:11:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0       

In [12]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

{'prompt': '<prompt text>', 'completion': '<ideal generated text>'}

In [13]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['dialogue']}" if sample["dialogue"] else None
    response = f"{RESPONSE_KEY}\n{sample['summary']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [14]:

#print(dataset)
#DatasetDict({
#    train: Dataset({
#        features: ['id', 'title', 'context', 'question', 'answers'],
#        num_rows: 130319
#    })
#    validation: Dataset({
#        features: ['id', 'title', 'context', 'question', 'answers'],
#        num_rows: 11873
#    })
#})

def create_prompt_formats_squad2(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Summarize the below conversation."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['question']}" if sample["question"] else None
    response = f"{RESPONSE_KEY}\n{sample['answers']['text']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [15]:
from datasets import load_dataset

# Load dataset from the hub

#huggingface_dataset_name = "neil-code/dialogsum-test"
#dataset = load_dataset(huggingface_dataset_name)
#print(dataset)

#dataset = load_dataset("b-mc2/sql-create-context", split="train")
#dataset = dataset.shuffle().select(range(12500))
# split dataset into 10,000 training samples and 2,500 test samples
#dataset = dataset.train_test_split(test_size=2500/12500)
# Convert dataset to OAI messages
#dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False) ##sql-dataset - "b-mc2/sql-create-context"

#print("Preprocessing dataset dialogsum_test")
#dataset_dialogsum_test = dataset.map(create_prompt_formats)#, batched=True) # for "neil-code/dialogsum-test"

# save datasets to disk
#dataset_dialogsum_test["train"].to_json("train_dataset_dialogsum.json", orient="records")
#dataset_dialogsum_test["test"].to_json("test_dataset_dialogsum.json", orient="records")
#print(dataset_dialogsum_test["train"][345])


print("Preprocessing dataset squad-2")

dataset = load_dataset("rajpurkar/squad_v2")
#dataset = dataset.shuffle()
dataset_squad2 = dataset.map(create_prompt_formats_squad2) #for "neil-code/dialogsum-test"
# save datasets to disk
dataset_squad2["train"].to_json("train_dataset_squad2.json", orient="records")
dataset_squad2["validation"].to_json("test_dataset_squad2.json", orient="records")




Preprocessing dataset squad-2


Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]

Map:   0%|          | 0/11873 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/131 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

16029984

In [16]:
print(dataset_squad2)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'text'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers', 'text'],
        num_rows: 11873
    })
})


In [17]:
from datasets import load_dataset

# Load jsonl data from disk for sql
#dataset_dialogsum_test = load_dataset("json", data_files="train_dataset_dialogsum.json", split="train")

dataset_squad2 = load_dataset("json", data_files="train_dataset_squad2.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [18]:
print(dataset_squad2)

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'text'],
    num_rows: 130319
})


In [19]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [ ]:
#print(dataset["train"][345]["messages"])
#dataset = load_dataset("rajpurkar/squad_v2")
#dataset = dataset.shuffle()

# save datasets to disk
#dataset["train"].to_json("train_dataset_squad2.json", orient="records")
#dataset["validation"].to_json("test_dataset_squad2.json", orient="records")


# Load jsonl data from disk for squad2
#dataset_squad2 = load_dataset("json", data_files="train_dataset_squad2.json", split="train")

In [20]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       2.0Gi        74Gi       2.0Mi       6.7Gi        80Gi
Swap:             0B          0B          0B


In [ ]:
!git lfs install

Git LFS initialized.


In [ ]:
!pip install git-lfs

In [ ]:
!git lfs install
!git lfs pull

In [46]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
#model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024 AND 08 MARCH 2024

#model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1" # 04/03/2024 NO MEMORY

model_id = "nvidia/OpenMath-Mistral-7B-v0.1-hf"

#model_id = "meta-llama/Llama-2-7b-hf" #08 march 2024 with "neil-code/dialogsum-test"

# meta-llama/Llama-2-7b-hf

#model_id = "abacusai/Smaug-72B-v0.1" #04 march 2024 and 08 march 2024 NO MEMORY
#model_id = "codellama/CodeLlama-7b-hf"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
model, tokenizer = setup_chat_format(model, tokenizer)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
text = "What is the capital of Canada?"

device = 'cuda'
model_inputs = tokenizer(text, return_tensors="pt").to(model.device)

generated_ids = model.generate(**model_inputs, temperature=0.9, top_k=1, top_p=1.0, repetition_penalty=1.4, min_new_tokens=16, max_new_tokens=128, do_sample=True)
decoded = tokenizer.decode(generated_ids[0])
print(decoded)

<s> What is the capital of Canada?
Ottawa. The city was chosen as a<|im_start|> for Parliament in 1857, and it became the permanent seat of government after Confederation in 1867. It has been home to federal institutions such as the Supreme Court since that time. Today Ottawa remains an important centre for politics<|im_start|> with many foreign embassies located here along with other international organizations like NATO headquarters or UNESCO offices among others. In addition there are several museums<|im_start|> dedicatedto Canadian history includingthe National GalleryofCanadaandTheMuseumofCivilizationwhich houses oneoftherichst collectionsin North America


In [48]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

MODEL GENERATION - ZERO SHOT

In [49]:
index=10
#dataset = dataset_dialogsum_test
#prompt = dataset[index]['dialogue']
#summary = dataset[index]['summary']

dataset = dataset_squad2
#features: ['id', 'title', 'context', 'question', 'answers'],
prompt = dataset[index]['question']
summary = dataset[index]['answers']['text']

original_model = model

#print(f"Index: {index}")
#print(f"Prompt: {prompt}")
#print(f"Summary: {summary}")

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
What was the first album Beyoncé released as a solo artist?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
['Dangerously in Love']

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Beyoncé released her first album as a solo artist in 2003, called Dangerously in Love.
The album was produced by Beyoncé's father, Matthew Knowles, and it includes the hits "Crazy in Love", "Baby Boy", and "Me, Myself, and I".
Beyoncé went on to release several more successful albums as a solo artist, including B'Day (2006), I Am


In [32]:
#print(len(dataset_dialogsum_test))

print(len(dataset_squad2))

130319


In [33]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
   

In [34]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [35]:
#!pip install transformers==4.36.2 --quiet
#!pip install transformers accelerate --quiet

from transformers import TrainingArguments

# frankmorales2020/Mistral-7B-text-to-sql
args = TrainingArguments(
    #output_dir="Mistral-7B-text-to-sql-flash-attention-2",    # directory to save and repository id
    #output_dir="Mixtral-8x7B-Instruct-v0.1-text-to-sql", # 04/03/2024
    # Llama-2-7b-hf
    #output_dir="Llama-2-7b-hf-text-to-sql-flash-attention-2", # 08/03/2024
    #output_dir="Llama-2-7b-hf-dialogsum-test-flash-attention-2", # 08/03/2024 #dataset_dialogsum_test

    #output_dir="OpenMath-Mistral-7B-v0.1-hf-dialogsum-test-flash-attention-2", # 08/03/2024 #dataset_dialogsum_test

    output_dir="OpenMath-Mistral-7B-v0.1-hf-squad2-flash-attention-2", # 09/03/2024 #dataset_squad2

    #output_dir="Smaug-72B-v0.1-text-to-sql"
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

Administration routines for huggingface.co

In [ ]:
#!pip install huggingface_hub --quiet
from huggingface_hub import HfApi

api = HfApi()
api.get_token_permission(token=access_token_write)
#api.set_access_token(access_token)
# frankmorales2020/Mistral-7B-text-to-sql Good


repo_id = "my-awesome-model-poc"
repo_id =  'frankmorales2020/Mistral-7B-squad2'
#frankmorales2020/Mistral-7B-squad2
#api.create_repo(repo_id=repo_id, private=False)

#api.delete_repo(repo_id=repo_id)


#api.upload_folder(
#    folder_path="./model",
#    repo_id=repo_id,
#    repo_type="model",
#)


https://huggingface.co/docs/trl/en/sft_trainer

In [36]:
#from trl import SFTTrainer

import trl

# explicitly import SFTTrainer
from trl.trainer import SFTTrainer


max_seq_length = 3072 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    #train_dataset=dataset,
    train_dataset=dataset_squad2,
    #train_dataset=dataset_dialogsum_test,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)


Generating train split: 0 examples [00:00, ? examples/s]

In [37]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,0.628200
20,0.473400
30,0.475700
40,0.485300
50,0.488200
60,0.466700
70,0.475000
80,0.465100
90,0.482900
100,0.490300


/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=

In [38]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

Test Model and run Inference

In [39]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
#peft_model_id = "./Mistral-7B-text-to-sql"
#/content/Llama-2-7b-hf-text-to-sql-flash-attention-2
#peft_model_id = "./Llama-2-7b-hf-text-to-sql-flash-attention-2"
#peft_model_id = "./Llama-2-7b-hf-dialogsum-test-flash-attention-2"

#peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-dialogsum-test-flash-attention-2"
peft_model_id = "./OpenMath-Mistral-7B-v0.1-hf-squad2-flash-attention-2"

#peft_model_id = "./Mixtral-8x7B-Instruct-v0.1-text-to-sql"
#peft_model_id = "./Smaug-72B-v0.1-text-to-sql" # 04/03/2024
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline

#KeyError: "Unknown task text-generationn, available tasks are ['audio-classification',
#'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering',
# 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction',
#'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation',
#'ner', 'object-detection', 'question-answering', 'sentiment-analysis',
#'summarization', 'table-question-answering', 'text-classification', 'text-generation',
#'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification',
#'translation', 'video-classification', 'visual-question-answering', 'vqa',
#'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification',
#'zero-shot-object-detection', 'translation_XX_to_YY']"

#Upon looking at the HuggingFacePipeline source code, I found it only supports below pipelines:
#text-generation`, text2text-generation and summarization

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

/content


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCau

In [40]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=256, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=256, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_fea

In [41]:
print(model_id)

nvidia/OpenMath-Mistral-7B-v0.1-hf


In [42]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen_after_tunning(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

MODEL GENERATION - AFTER TUNNING

In [45]:
index=10
dataset = dataset_squad2
TUNE_model = model

## dialogsum_test
#prompt = dataset[index]['dialogue']
#summary = dataset[index]['summary']

## squad2.0
#features: ['id', 'title', 'context', 'question', 'answers'],
prompt = dataset[index]['question']
summary = dataset[index]['answers']['text']

formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen_after_tunning(TUNE_model,formatted_prompt,1024,)
#print(res[0])
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - AFTER THE TUNNING:\n{output}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Instruct: Summarize the following conversation.
What was the first album Beyoncé released as a solo artist?
Output:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
['Dangerously in Love']

---------------------------------------------------------------------------------------------------
MODEL GENERATION - AFTER THE TUNNING:
['Dangerously in Love']

### End 
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruct: Summarize the following conversation.

What artist did Beyoncé date?

### 


In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

Deploy the LLM for Production WITH dockers

In [ ]:
%%shell
pip install udocker
udocker --allow-root install

In [ ]:
%%shell

sudo apt update -qq
sudo apt install apt-transport-https ca-certificates curl software-properties-common -qq
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -
sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu bionic stable"
sudo apt update -qq
sudo apt install docker-ce
docker

In [ ]:
%%shell

docker --version

In [ ]:
%%shell
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker info
docker network ls
docker pull hello-world
docker pull ubuntu
# docker build -t myimage .
docker images
kill $(jobs -p)

In [ ]:
%%shell

docker --version

model=/content/Mistral-7B-text-to-sql
num_shard=1             # number of shards
max_input_length=1024   # max input length
max_total_tokens=2048   # max total tokens

docker run -d --name tgi --gpus all -ti -p 8080:80 \
  -e MODEL_ID=/workspace \
  -e NUM_SHARD=$num_shard \
  -e MAX_INPUT_LENGTH=max_input_length \
  -e MAX_TOTAL_TOKENS=max_total_tokens \
  -v $model:/workspace \
  ghcr.io/huggingface/text-generation-inference:latest


Once your container is running you can send requests.

In [ ]:
import requests as r
from transformers import AutoTokenizer
from datasets import load_dataset
from random import randint

# Load our test dataset and Tokenizer again
#tokenizer = AutoTokenizer.from_pretrained("Mistral-7B-text-to-sql")
tokenizer = AutoTokenizer.from_pretrained("Mistral-7B-text-to-sql-flash-attention-2")
#tokenizer = AutoTokenizer.from_pretrained("Smaug-72B-v0.1")
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# generate the same prompt as for the first local test
prompt = tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
request= {"inputs":prompt,"parameters":{"temperature":0.2, "top_p": 0.95, "max_new_tokens": 256}}

# send request to inference server
resp = r.post("http://127.0.0.1:8080/generate", json=request)

print(resp)
output = resp.json()["generated_text"].strip()
time_per_token = resp.headers.get("x-time-per-token")
time_prompt_tokens = resp.headers.get("x-prompt-tokens")

# Print results
print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{output}")
print(f"Latency per token: {time_per_token}ms")
print(f"Latency prompt encoding: {time_prompt_tokens}ms")

Awesome, Don't forget to stop your container once you are done.

In [ ]:
!docker stop tgi

Deploy the LLM for Production WITH huggingface.co
https://github.com/frank-morales2020/MLxDL/blob/main/upload_model_hf.ipynb